# Understand instances and samples structure

In [16]:
import gzip, pickle
import torch
import numpy as np

# with gzip.open("data/samples/facilities/100_100_5/test/sample_3.pkl", "rb") as f:
# with gzip.open("data/samples/setcover/500r_1000c_0.05d/valid/sample_13564.pkl", "rb") as f:
with gzip.open("data/samples/setcover/500r_1000c_0.05d/test/sample_180.pkl", "rb") as f:
    sample = pickle.load(f)

In [17]:
sample

{'episode': 18,
 'instance': np.str_('data/instances/setcover/test_500r_1000c_0.05d/instance_671.lp'),
 'seed': 1381534676,
 'node_number': 937,
 'node_depth': 16,
 'data': [({'names': ['obj_cosine_similarity',
     'bias',
     'is_tight',
     'age',
     'dualsol_val_normalized'],
    'values': array([[-2.14891285e-01, -1.52498573e-01,  1.00000000e+00,
             8.65051903e-04,  0.00000000e+00],
           [-1.95415437e-01, -1.41421363e-01,  0.00000000e+00,
             8.65051903e-04,  0.00000000e+00],
           [-2.30558351e-01, -1.27000123e-01,  0.00000000e+00,
             7.78546713e-03,  0.00000000e+00],
           ...,
           [-6.65286660e-01, -4.09258544e-01,  0.00000000e+00,
             4.05709343e-01, -0.00000000e+00],
           [-6.11399114e-01, -5.47798395e-01,  0.00000000e+00,
             4.05709343e-01, -0.00000000e+00],
           [-7.01349914e-01, -4.64603156e-01,  0.00000000e+00,
             4.03979239e-01, -0.00000000e+00]], shape=(520, 5))},
   {'names

In [18]:
instance_file = str(sample["instance"])
with open(instance_file, "r") as f:
    instance_lp = f.read()
print(instance_lp)

minimize
OBJ: +74 x1 +88 x2 +29 x3 +15 x4 +42 x5 +71 x6 +20 x7 +61 x8 +53 x9 +1 x10 +56 x11 +31 x12 +60 x13 +89 x14 +34 x15 +4 x16 +88 x17 +11 x18 +96 x19 +54 x20 +39 x21 +76 x22 +12 x23 +13 x24 +90 x25 +60 x26 +65 x27 +28 x28 +48 x29 +59 x30 +98 x31 +73 x32 +38 x33 +100 x34 +81 x35 +57 x36 +30 x37 +39 x38 +61 x39 +85 x40 +87 x41 +59 x42 +48 x43 +70 x44 +21 x45 +15 x46 +43 x47 +58 x48 +75 x49 +82 x50 +75 x51 +18 x52 +69 x53 +17 x54 +75 x55 +62 x56 +62 x57 +76 x58 +38 x59 +5 x60 +9 x61 +26 x62 +65 x63 +90 x64 +51 x65 +42 x66 +37 x67 +100 x68 +96 x69 +28 x70 +13 x71 +54 x72 +76 x73 +3 x74 +30 x75 +43 x76 +57 x77 +65 x78 +6 x79 +36 x80 +38 x81 +39 x82 +56 x83 +58 x84 +78 x85 +94 x86 +19 x87 +21 x88 +88 x89 +39 x90 +10 x91 +48 x92 +93 x93 +88 x94 +14 x95 +86 x96 +91 x97 +55 x98 +52 x99 +41 x100 +77 x101 +8 x102 +2 x103 +10 x104 +56 x105 +24 x106 +2 x107 +21 x108 +16 x109 +71 x110 +93 x111 +64 x112 +33 x113 +45 x114 +19 x115 +75 x116 +54 x117 +35 x118 +93 x119 +27 x120 +15 x121 +6 x122 +39 

In [19]:
constraint_dict, edge_dict, variable_dict = sample["data"][0]

variable_names = variable_dict["names"]
variable_feature_indices = {name: variable_names.index(name) for name in variable_names}
variable_default_features = torch.as_tensor(variable_dict["values"], dtype=torch.float32)

constraint_names = constraint_dict["names"]
constraint_feature_indices = {name: constraint_names.index(name) for name in constraint_names}
constraint_default_features = torch.as_tensor(constraint_dict["values"], dtype=torch.float32)
# pick problem data features
constraint_required = ["bias", "dualsol_val_normalized"]
variable_required = ["type_0", "type_1", "type_2", "type_3", 
                     "has_lb", "has_ub", "sol_is_at_lb", "sol_is_at_ub", "sol_frac",
                     "coef_normalized", "sol_val"]

use_default_features = False
if use_default_features:
    variable_features = variable_default_features
    constraint_features = constraint_default_features
else:
    variable_required = ["type_0", "type_1", "type_2", "type_3", 
                                 "has_lb", "has_ub", "sol_is_at_lb", "sol_is_at_ub", "sol_frac",
                                 "coef_normalized", "sol_val"]
    constraint_required = ["bias", "dualsol_val_normalized"]
    variable_features = torch.stack([variable_default_features[:, variable_feature_indices[name]]
                                    for name in variable_required], 
                                    axis=-1)
    constraint_features = torch.stack([constraint_default_features[:, constraint_feature_indices[name]]
                                    for name in constraint_required], 
                                    axis=-1)
    variable_feature_indices = {name: new_index for name, new_index in zip(variable_required, range(len(variable_required)))}
    constraint_feature_indices = {name: new_index for name, new_index in zip(constraint_required, range(len(constraint_required)))}
    
# add candidate indicator to variable features
candidate_choice_node_id = sample["data"][2]
candidates = torch.as_tensor(sample["data"][3], dtype=torch.int64)
candidate_scores = torch.as_tensor(sample["data"][4], dtype=torch.float32)
candidate_choice = torch.where(candidates == torch.as_tensor(sample["data"][2], dtype=torch.int64))[0][0]

In [20]:
def clean_candidates(candidates, candidate_scores, candidate_choice_node_id, 
                     variable_features, variable_feature_indices):
    sol_is_not_at_lb = variable_features[candidates, variable_feature_indices["sol_is_at_lb"]] == 0
    sol_is_not_at_ub = variable_features[candidates, variable_feature_indices["sol_is_at_ub"]] == 0
    sol_is_not_at_lub = sol_is_not_at_lb & sol_is_not_at_ub
    cleaned_candidates = candidates[sol_is_not_at_lub]
    cleaned_candidate_scores = candidate_scores[sol_is_not_at_lub]
    if cleaned_candidates.numel() < 1:
        raise ValueError("no candidate exists after cleaning")
    if candidate_choice_node_id not in cleaned_candidates:
        print(f"Rare case: all candidates are equally bad (as bad as nodes whose LP solution is at LB/UB.) This usually happens when branching on 'real candidate' produces the same optimal obj value.")
        print(f"Replace the candidate choice with a new one in the cleaned candidates.")
        new_candidate_choice = cleaned_candidate_scores.argmax().item()
        new_candidate_choice_node_id = cleaned_candidates[new_candidate_choice].item()
        candidate_choice_node_id = new_candidate_choice_node_id
    return cleaned_candidates, cleaned_candidate_scores, candidate_choice_node_id
    

In [21]:
sol_is_not_at_lb = variable_features[candidates, variable_feature_indices["sol_is_at_lb"]] == 0
sol_is_not_at_ub = variable_features[candidates, variable_feature_indices["sol_is_at_ub"]] == 0
sol_is_not_at_lub = sol_is_not_at_lb & sol_is_not_at_ub
sol_is_not_at_lub

tensor([False,  True, False,  True,  True,  True, False, False,  True,  True,
         True,  True, False,  True, False, False,  True, False,  True, False,
         True,  True, False,  True, False,  True,  True,  True, False,  True,
         True, False,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True, False, False, False,  True,  True,  True,
        False,  True, False,  True,  True, False,  True, False,  True,  True,
         True,  True, False, False,  True, False])

In [22]:
candidate_scores

tensor([1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10,
        1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10,
        1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10,
        1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10,
        1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10,
        1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10,
        1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10,
        1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10,
        1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10,
        1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10,
        1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10])

In [23]:
variable_features[candidates, variable_feature_indices["sol_is_at_lb"]] == 0

tensor([ True,  True, False,  True,  True,  True, False, False,  True,  True,
         True,  True, False,  True, False,  True,  True, False,  True, False,
         True,  True, False,  True,  True,  True,  True,  True, False,  True,
         True, False,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True, False, False, False,  True,  True,  True,
         True,  True,  True,  True,  True, False,  True, False,  True,  True,
         True,  True,  True, False,  True, False])

In [24]:
candidates, candidate_scores, candidate_choice_node_id = clean_candidates(candidates, candidate_scores,
                                                                          candidate_choice_node_id, variable_features, variable_feature_indices)

Rare case: all candidates are equally bad (as bad as nodes whose LP solution is at LB/UB.) This usually happens when branching on 'real candidate' produces the same optimal obj value.
Replace the candidate choice with a new one in the cleaned candidates.


In [25]:
candidates

tensor([616, 313, 994, 822, 924,  73, 137, 478, 361,   3, 534, 206, 818, 195,
        698, 428, 106, 580, 953, 228, 547, 977,  17, 874, 597, 771, 405, 274,
        982, 102, 182, 751, 251, 386, 773, 440, 391, 208, 336,  90,  59, 708,
        628])

In [26]:
candidates_feature = torch.zeros(variable_features.size(0), dtype=torch.float32)
candidates_feature[candidates] = 1.0
variable_features = torch.cat([variable_features, candidates_feature.unsqueeze(-1)], dim=-1)

In [27]:
print(len(candidates), variable_features[:, -1].sum())

43 tensor(43.)


In [28]:
int_mask = variable_features[:, variable_feature_indices["type_0"]] == 1
binary_mask = variable_features[:, variable_feature_indices["type_1"]] == 1
continuos_mask = variable_default_features[:, variable_feature_indices["type_3"]] == 1
sol_at_lb_mask = variable_features[:, variable_feature_indices["sol_is_at_lb"]] == 1
sol_at_ub_mask = variable_features[:, variable_feature_indices["sol_is_at_ub"]] == 1
print(f"# candidates: {len(candidates)}")
print(f"# int: {int_mask.sum().item() }")
print(f"# binary: {binary_mask.sum().item() }")
print(f"# continuous: {continuos_mask.sum().item() }")
print(f"# sol at lb: {sol_at_lb_mask.sum().item() }")
print(f"# sol at ub: {sol_at_ub_mask.sum().item() }")
int_sol_at_lb_mask = int_mask & sol_at_lb_mask
int_sol_at_ub_mask = int_mask & sol_at_ub_mask
print(f"# int sol at lb: {int_sol_at_lb_mask.sum().item() }")
print(f"# int sol at ub: {int_sol_at_ub_mask.sum().item() }")
candidate_mask = variable_features[:, -1] == 1
print(f"# candidate & int: {(candidate_mask & int_mask).sum().item() }")

# candidates: 43
# int: 1000
# binary: 0
# continuous: 0
# sol at lb: 951
# sol at ub: 940
# int sol at lb: 951
# int sol at ub: 940
# candidate & int: 43


In [29]:
variable_features[candidates, variable_feature_indices["sol_val"]]

tensor([0.1429, 0.1429, 0.4286, 0.8571, 0.8571, 0.7143, 0.1429, 0.1429, 0.1429,
        0.1429, 0.1429, 0.8571, 0.8571, 0.8571, 0.5714, 0.2857, 0.1429, 0.1429,
        0.1429, 0.7143, 0.8571, 0.1429, 0.2857, 0.1429, 0.8571, 0.1429, 0.8571,
        0.1429, 0.7143, 0.7143, 0.8571, 0.1429, 0.8571, 0.1429, 0.1429, 0.8571,
        0.7143, 0.5714, 0.8571, 0.7143, 0.2857, 0.7143, 0.8571])

In [30]:
print(f"# candidates: {len(candidates)}")
int_mask = (variable_features[:, variable_feature_indices["type_0"]] == 1)
sol_at_lb_mask = (variable_features[:, variable_feature_indices["sol_is_at_lb"]] == 1)
sol_at_ub_mask = (variable_features[:, variable_feature_indices["sol_is_at_ub"]] == 1)
(sol_at_lb_mask & sol_at_ub_mask).sum()

# candidates: 43


tensor(934)

In [31]:
constraint_names = sample["data"][0][0]["names"]
for i in range(len(constraint_names)):
    name = constraint_names[i]
    print(f"{name}: {i}")

obj_cosine_similarity: 0
bias: 1
is_tight: 2
age: 3
dualsol_val_normalized: 4


In [32]:
variable_names = sample["data"][0][2]["names"]
for i in range(len(variable_names)):
    name = variable_names[i]
    print(f"{name}: {i}")

type_0: 0
type_1: 1
type_2: 2
type_3: 3
coef_normalized: 4
has_lb: 5
has_ub: 6
sol_is_at_lb: 7
sol_is_at_ub: 8
sol_frac: 9
basis_status_0: 10
basis_status_1: 11
basis_status_2: 12
basis_status_3: 13
reduced_cost: 14
age: 15
sol_val: 16
inc_val: 17
avg_inc_val: 18
